<div class="alert alert-block alert-success">
<h3> Exercise 1: Solve the balls and bin problem via an integer program</h3>
<p>Assume that you have the data for balls and bins provided to you.</p>
<p>Solve to find the best assignment of balls into bins</p>
</div>

<img src="imgs/Balls.png" width="60%">

<img src="imgs/Bins.png" width="60%">

### Rules
<ol>
<li>Putting a ball into a bin gives you a reward equal to their product (ball score X bin score).
<li>Balls can go into bins of matching colors only.
<li>Small bin can accept only one small ball.
<li> Big bin can accept either one large ball or two small balls.
</ol>

<div class="alert alert-block alert-success">
<h3> Exercise 2: Trading off different users</h3>
<p>Assume that you have two different user groups that use the menu in very different ways, e.g. novice versus expert users. </p>
<p>Given two sets of frequency distributions for the menu items, $p^{novice}$ and $p^{expert}$ your task is to reformulate the objective function so that it finds the best design for both user groups. </p>
</div>

In [10]:
from gurobipy import *
import utils_anna

def solve(elements, positions, frequency, distance):
    # ==== 1. Create the (empty) model ====
    model = Model("linear_menu")

    # ==== 2. Add decision variables ======
    x = {}
    # Create one binary variable for each element-position pair. 
    # We give it a meaningful name so we later understand what it means
    # if it is set to 1
    for e in elements:
        for p in positions:
            x[(e,p)] = model.addVar(vtype=GRB.BINARY, name="%s_%i"%(e,p))            
    # Integrate new variables
    model.update()
    
    # ====3. Add Constraints ======
    # Add constraints
    # Each position is only assigned to one element
    for p in positions: 
        model.addConstr(quicksum(x[(e,p)]
                   for e in elements) == 1, "uniqueness_constraint_%i"%p)    
    # Each element is only assigned to one position
    for e in elements: 
        model.addConstr(quicksum(x[(e,p)]
                   for p in positions) == 1, "uniqueness_constraint_%s"%e)
    model.update()
    
    # ==== 4. Specify Objective function ======
    reading_cost = 0.4 # assumed that scanning a single item takes 400 ms
    
    # Sum up the costs for mapping any element e to any position p 
    cost = quicksum(frequency[e] * distance[p] * reading_cost * x[(e,p)]
                                for e in elements
                                   for p in positions)
    model.setObjective(cost,GRB.MINIMIZE)

    
    # ==== 5. Optimize model ======    
    model.optimize()
    
    # ====6. Extract solution ======   
    layout = [None]*len(elements)
    #create the layout (ordered list of elements) from the variables 
    #that are set to 1
    for v in model.getVars():
        if v.x == 1:
            element = v.varName.split("_")[0]        
            position = int(v.varName.split("_")[1])
            layout[position] = element                 
    
    return layout, model.getObjective().getValue()
    
    

In [11]:
#define elements and positions
#elements = ['Open', 'About','Quit','Help','Close',
#            'Save','Edit','Insert','Delete']
elements = ['item1-1', 'item1-2', 'item1-3', 'item1-4', 'item2-1', 'item2-2', 'item2-3', 'item2-4',
            'item3-1', 'item3-2', 'item3-3', 'item3-4', 'item4-1', 'item4-2', 'item4-3', 'item4-4']
positions = list(range(len(elements)))

In [12]:
#define cost factors
#frequency = {'Quit':0.004,'About':0.2,'Open':0.1,'Save':0.1,'Close':0.05,
#             'Help':0.02,'Edit':0.08,'Insert':0.1,'Delete':0.05}
frequency = {'item1_1':0.3,'item1_2':0.2,'item1_3':0.1,'item1_4':0.1,
             'item2_1':0.3,'item2_2':0.2,'item2_3':0.1,'item2_4':0.1,
             'item3_1':0.3,'item3_2':0.2,'item3_3':0.1,'item3_4':0.1,
             'item4_1':0.3,'item4_2':0.2,'item4_3':0.1,'item4_4':0.1}
distance = map(lambda p:utils_anna.distance(1,0,p), positions)

In [13]:
#solve the problem
layout, objective = solve(elements, positions, frequency, distance)
 
#Print the solution
print "Objective value (expected selection time):", objective
#plot the layout
print layout
#SVGlayout = render_anna.SVGlayout(layout,1)
#SVG(SVGlayout.inSVG)

KeyError: 'item1-1'


<div class="alert alert-block alert-success">
<h3> Exercise 3a: Modeling the letter assignment problem</h3>
How can you model the problem of assigning characters to keyslots on a keyboard mathematically?
<ol>
<li> Define the decision variables
<li> Add the necessary constraints
<li> Formulate the objective function. 
</ol>
</div>

<br>
<div class="alert alert-block alert-success">
<h3> Exercise 3b: Implementing the letter assignment problem in Gurobi</h3>
Implement the model in Gurobi and optimize a keyboard layout for the given letters.
</div>

<div class="alert alert-block alert-success">
<h3> Bonus task:</h3>
<p>We really want to name our keyboard the "HCI" keyboard.</p>
<br>
<p>Therefore, your task is to change the mathematical model and its implementation so that the letters H - C - I are placed next to each other on any of the rows of the computer, as in the example keyboard below. Do not change the input data.</p>
<p>
How much worse is this keyboard in comparison to the unconstrained problem?</p>                                                 
</div>